# Import Libraries

In [1]:
from __future__ import print_function, division, absolute_import
import os,time
import numpy as np
np.random.seed(1)
import math
import utils
from utils import Scoring
import matplotlib.pyplot as plt
from dataLoader import reshape_folds
from collections import Counter
import pandas as pd
import argparse

import torch
torch.cuda.empty_cache()
from torch.autograd import Variable
import torch.nn as nn
from torch.backends import cudnn

# List of Input Features

In [2]:
df = pd.read_excel (r'../logs/pretrained.xlsx')
input_features = df["feature"].iloc[:]
input_features

0                      fbank
1                  log-fbank
2                    mfcc_26
3                    mfcc_13
4            fbank_log-fbank
5              fbank_mfcc_13
6          log-fbank_mfcc_13
7    fbank_log-fbank_mfcc_13
8                  mfcc_13_d
9                 mfcc_13_dd
Name: feature, dtype: object

# Get Results From Two Different Input Features

Select two features from the upper list to observe the validation result and put them in the "Selected Feature List" below.

In [3]:
feature_name = []
predictions = []

for feature in ["fbank", "log-fbank"]: # Selected Feature List
    
    index = [i for i, feat in enumerate(input_features) if(feat==feature)][0]

    class paramClass():
        def __init__(self):
            self.epoch = df["epoch"][index]
            self.batch_size = df["batch_size"][index]
            self.inp = df["inp"][index]
            self.base_lr = df["base_lr"][index]
            self.max_lr = df["max_lr"][index]
            self.step_size = df["step_size"][index]
            self.checkpoint = '../logs/' + self.inp + "/checkpoints/"
    param = paramClass()

    data_file = "../data/feature/train_val_npzs/" + param.inp + "_train_val_data.npz"
    data = np.load(data_file)

    x_val = data['x_val_mfcc']
    y_val = data['y_val']
    val_parts = data['val_parts']
    val_wav_files = data['val_wav_files']

    x_val = x_val.reshape(x_val.shape[0],x_val.shape[1],x_val.shape[2], 1)

    #--- Reshape the folds
    _, [y_val] = reshape_folds([], [y_val])

    from keras.utils import to_categorical
    y_val = to_categorical(y_val, num_classes = 2)

    from model import resnet_extractor, abn_classifier
    nodes = [16, 32, 64, 128]
    num_layers = 2
    model_fe = resnet_extractor(1, None, param.inp, nodes, num_layers).cuda()
    model_abn = abn_classifier(model_fe.out_features, 2, dropout = None).cuda()

    e = param.epoch - 1
    checkpoint_fe = param.checkpoint + str(e+1) + "_saved_fe_model.pth.tar"
    model_fe.load_state_dict(torch.load(checkpoint_fe)["state_dict"])
    checkpoint_abn = param.checkpoint + str(e+1) + "_saved_cls_model.pth.tar"
    model_abn.load_state_dict(torch.load(checkpoint_abn)["state_dict"])

    epoch_loss = torch.load(checkpoint_fe)["loss"]
    val_loss_load = torch.load(checkpoint_fe)["val_loss"]

    loss_fn = nn.CrossEntropyLoss()

    print("\n" + feature + " --- EPOCH ---- ( " + str(e+1) + " )----")

    model_fe.eval()
    model_abn.eval()

    with torch.no_grad():
        cls_pred = None
        cls_val = None
        total_features = np.array([])
        epoch_val_loss = 0
        s = 0
        for i, part in enumerate(val_parts):
            x,y = torch.from_numpy(x_val[s:s+part]),torch.from_numpy(y_val[s:s+part])
            s = s + part

            if(len(x) == 0): # If no bits are found
                continue

            x,y = Variable(x),Variable(y)
            x = x.type(torch.FloatTensor).cuda()

            x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2])
            y = torch.tensor(np.array(y).reshape(-1, 2)).cuda().float()

            features = model_fe(x)
            cls = model_abn(features)

            val_loss = loss_fn(cls, torch.argmax(y,dim=1))

            if(i==0): total_features = features.cpu().numpy()
            else: total_features = np.concatenate((total_features, features.cpu().numpy()))

            if(cls_pred is None):
                cls_pred = cls
                cls_val = y
            else:
                cls_pred = torch.cat((cls_pred,cls))
                cls_val = torch.cat((cls_val,y))

            epoch_val_loss = epoch_val_loss + val_loss

        epoch_val_loss = epoch_val_loss/len(val_parts)

        print("Validation loss - ", str(epoch_val_loss.item()))

    domain = np.asarray(val_wav_files).reshape((-1, 1))
    score_log = Scoring(e + 1)
    _, true, pred, _ = score_log.log_score(cls_pred, cls_val, val_parts, 
            y_domain = domain, list_out=True)
    
    feature_name.append(feature)
    predictions.append(pred)


fbank --- EPOCH ---- ( 25 )----
Validation loss -  0.5178222060203552
TN: 122 FP: 24 FN: 25 TP: 113
Sensitivity: 0.819 Specificity: 0.836 Precision: 0.825 F1: 0.822 MACC 0.827 Accuracy 0.827

log-fbank --- EPOCH ---- ( 64 )----
Validation loss -  0.5149890184402466
TN: 121 FP: 25 FN: 19 TP: 119
Sensitivity: 0.862 Specificity: 0.829 Precision: 0.826 F1: 0.844 MACC 0.846 Accuracy 0.845


# McNemer Test

In [4]:
from statsmodels.stats.contingency_tables import mcnemar
from mlxtend.evaluate import mcnemar_table
def McnemerStats(true,pred1,pred2,threshold = 0.05, lab1='model 1', lab2='model 2'):
    """
    true: true labels of the data. 
    pred1: predicted values from model 1 ## generated from block 8
    pred2: predicted values from model 2 ## generated from block 8
    """
    pred1 = 1*(np.asarray(pred1) > 0.5)
    pred2 = 1*(np.asarray(pred2) > .5)
    true = np.asarray(true)
    
    mc_table = mcnemar_table(y_target=true, 
                   y_model1=pred1, 
                   y_model2=pred2)
    if(np.min(mc_table)<25):
        result = mcnemar(mc_table, exact=True)
    else:
        result = mcnemar(mc_table, exact=False, correction=True)
    print('statistic=%.10f, p-value=%.10f' % (result.statistic, result.pvalue))
    # interpret the p-value
    alpha = threshold
    if result.pvalue > alpha:
        print('Same proportions of errors, models make similar error (fail to reject H0)')
    else:
        print('Different proportions of errors, error rates are different (reject H0)')
    return result.statistic,result.pvalue

In [5]:
McnemerStats(true,predictions[0],predictions[1])

statistic=18.0000000000, p-value=0.5327092552
Same proportions of errors, models make similar error (fail to reject H0)


(18.0, 0.5327092552361133)